This is a notebook to examine annotations for spontaneous behaviors.  It is not involved in processing data for any publication results, but is meant only for data exploration. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import itertools
from pathlib import Path

import pandas as pd

from keller_zlatic_vnc.data_processing import apply_quiet_and_cutoff_times
from keller_zlatic_vnc.data_processing import generate_standard_id_for_full_annots
from keller_zlatic_vnc.data_processing import get_basic_clean_annotations_from_full
from keller_zlatic_vnc.data_processing import read_full_annotations
from keller_zlatic_vnc.data_processing import count_transitions

/Users/bishopw/Documents/Janelia_Research/Projects/janelia_core/janelia_core/fileio/exp_reader.py:21: UserWarning: Unable to locate pyklb module.  Will not be able to read in .klb files.
  warnings.warn('Unable to locate pyklb module.  Will not be able to read in .klb files.')


## Parameters go here 

In [3]:
ps = dict()

# Give locations of folders with annotations
ps['annot_folders'] = [r'/Volumes/bishoplab/projects/keller_vnc/data/full_annotations/em_volume_behavior_csv']

# Specify the definition for clean events we use 
ps['clean_event_def'] = 'po'

# Specify the threshold we use (in number of stacks) to determine when a quiet transition has occurred
ps['q_th'] = 21

# Specify the cut off threshold we use (in number of stacks) to determine when a real transition has occurred
ps['co_th'] = 3

# Specify the length of events we mark new quiet events as 
ps['q_length'] = 3

# Specify the acceptable preceding behaviors for events we analyze
ps['pre_behs'] = ['Q', 'B', 'F', 'TL', 'TR', 'H']

# Specify the set of acceptable behaviors transitioned into for events we analyze
ps['behs'] = ['B', 'F', 'TL', 'TR', 'H']

## Get list of all subjects we have annotations for 

In [4]:
annot_file_paths = list(itertools.chain(*[glob.glob(str(Path(folder) / '*.csv')) for folder in ps['annot_folders']]))
annot_file_names = [Path(p).name for p in annot_file_paths]
annot_subjs = [generate_standard_id_for_full_annots(fn) for fn in annot_file_names]


## Read in basic annotations for all subjects 

In [5]:
annotations = []
for s_id, s_annot_file in zip(annot_subjs, annot_file_paths):
    tbl = read_full_annotations(s_annot_file)
    tbl['subject_id'] = s_id
    annotations.append(tbl)

## Filter down to only "clean" events 

In [6]:
annotations = [get_basic_clean_annotations_from_full(annot, clean_def=ps['clean_event_def'])
                   for annot in annotations]

## Put all events into the same table 

In [7]:
annotations = pd.concat(annotations, ignore_index=True)

## Mark preceding and succeeding behaviors as known, quiet or unknown 

In [8]:
annotations = apply_quiet_and_cutoff_times(annotations, quiet_th=ps['q_th'], 
                                           co_th=ps['co_th'], q_length=ps['q_length'])

## See how many events we have of each type of transition

In [9]:
print('Number of events before filtering: ' + str(len(annotations)))

Number of events before filtering: 398


In [10]:
trans_tbl = count_transitions(annotations, before_str='beh_before', after_str='beh')
trans_tbl.insert(9, 'Sum', trans_tbl.sum(axis=1))
trans_tbl.loc['Sum'] = trans_tbl.sum(axis=0)

In [11]:
trans_tbl

,B,F,H,O,P,Q,TL,TR,U,Sum
B,0.0,0.0,4.0,0.0,0.0,0.0,8.0,6.0,0.0,18.0
F,0.0,5.0,0.0,0.0,41.0,0.0,4.0,2.0,0.0,52.0
H,2.0,0.0,0.0,0.0,2.0,0.0,2.0,3.0,0.0,9.0
O,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0
P,0.0,72.0,1.0,0.0,2.0,0.0,2.0,1.0,0.0,78.0
Q,4.0,9.0,2.0,1.0,6.0,0.0,1.0,3.0,0.0,26.0
TL,6.0,0.0,1.0,0.0,2.0,0.0,2.0,3.0,0.0,14.0
TR,11.0,0.0,2.0,0.0,1.0,0.0,2.0,5.0,0.0,21.0
U,34.0,53.0,7.0,1.0,47.0,0.0,16.0,19.0,0.0,177.0
Sum,57.0,140.0,17.0,2.0,103.0,0.0,37.0,42.0,0.0,398.0


## Filter out events not starting or ending in an acceptable behavior

In [12]:
if ps['behs'] is not None:
    keep_inds = [i for i in annotations.index if annotations['beh'][i] in ps['behs']]
    annotations = annotations.loc[keep_inds]

if ps['pre_behs'] is not None:
    keep_inds = [i for i in annotations.index if annotations['beh_before'][i] in ps['pre_behs']]
    annotations = annotations.loc[keep_inds]


## Now see how many events we have of each type of transition

In [13]:
print('Number of events after filtering: ' + str(len(annotations)))

Number of events after filtering: 87


In [14]:
trans_tbl = count_transitions(annotations, before_str='beh_before', after_str='beh')
trans_tbl.insert(6, 'Sum', trans_tbl.sum(axis=1))
trans_tbl.loc['Sum'] = trans_tbl.sum(axis=0)

In [15]:
trans_tbl

,B,F,H,Q,TL,TR,Sum
B,0.0,0.0,4.0,0.0,8.0,6.0,18.0
F,0.0,5.0,0.0,0.0,4.0,2.0,11.0
H,2.0,0.0,0.0,0.0,2.0,3.0,7.0
Q,4.0,9.0,2.0,0.0,1.0,3.0,19.0
TL,6.0,0.0,1.0,0.0,2.0,3.0,12.0
TR,11.0,0.0,2.0,0.0,2.0,5.0,20.0
Sum,23.0,14.0,9.0,0.0,19.0,22.0,87.0
